In [3]:
import os 
os.chdir("..")  # Navigate to the parent directory to access src module

In [ ]:
import torch
from src.model import ResNet

# Path to your saved model
# model_path = "/home/zakarianarjis/workspace/recursive-reasoning/experiments/ResNet_CIFAR100_2025-11-17_15-52-31/best_model.pth" # recursive mode false 
model_path = "/home/zakarianarjis/workspace/recursive-reasoning/experiments/ResNet_CIFAR100_2025-11-14_21-30-45/best_model.pth" # recursive mode true Nsup 8 , Nlatent 6, Ndeep 3
# Create model with the right arguments (adjust if needed)
model = ResNet(
    num_classes=100,
    recursive_mode=True,
    pretrained=True,
    use_precomputed_features=True
)

# Load weights
state_dict = torch.load(model_path, map_location="cuda" )
model.load_state_dict(state_dict)
model.to("cuda")

model.eval()
print("Model loaded successfully!")


Model loaded successfully!


In [111]:
import torch

path = "/scratch/narjis/CIFAR100/precomputed_embeding/train.pt"

data = torch.load(path)      # load the tensor/list/dict
one_feature = data['features'][0].reshape(1, -1)  # get the first feature and reshape to (1, feature_dim)
embed_out = model.get_input_embedding(one_feature.cuda())

In [126]:
output_1 = model.backbone(2*embed_out)
output_2 = model.backbone(embed_out)
output_1/output_2

tensor([[   nan, 1.7717, 1.9903, 1.8971, 1.9694,    nan, 1.9483,    nan,    nan,
         1.9555, 1.9457,    nan, 1.8154,    nan, 1.9813,    nan, 1.6178, 1.8746,
            nan,    nan,    nan,    nan,    nan, 1.9893, 1.9451,    nan,    nan,
            nan, 1.8789,    nan, 1.8848,    nan,    nan,    nan,    nan,    nan,
            nan, 1.9757, 0.9263,    nan, 1.8442, 0.0000,    nan, 1.6641, 1.9921,
            nan,    nan,    nan, 1.7138, 1.9808, 1.9972,    nan, 1.9335, 1.9805,
            nan, 1.9783, 1.9570, 1.8617,    nan, 1.8871, 1.9678,    nan, 1.9846,
            nan,    nan, 2.0676,    nan, 1.8821,    nan, 1.9815, 1.9599, 1.9596,
         1.9669,    nan,    nan,    nan,    nan,    nan, 1.9880, 1.9735, 2.0854,
            nan,    nan,    nan,    nan, 1.9458,    nan,    nan, 1.9836, 1.9318,
            nan, 2.5214,    nan,    nan,    nan, 1.9867,    nan, 2.0012,    nan,
            nan,    nan,    nan, 1.9754, 1.9816,    nan,    nan, 1.8740, 1.9644,
            nan, 1.9924, 1.9

In [95]:
import torch
import torch.nn as nn

def check_linear_behavior(model, input_dim, num_samples=5000):
    model.eval()
    x = torch.rand(num_samples, input_dim)
    with torch.no_grad():
        y = model(x)

    # Fit a linear model: y ≈ Ax + b
    linear = nn.Linear(input_dim, y.shape[1], bias=True)
    optimizer = torch.optim.LBFGS(linear.parameters(), max_iter=500)

    def closure():
        optimizer.zero_grad()
        pred = linear(x)
        loss = ((pred - y)**2).mean()
        loss.backward()
        return loss

    optimizer.step(closure)

    # Evaluate how well the linear layer approximates the MLP
    with torch.no_grad():
        pred = linear(x)
        error = ((pred - y)**2).mean().sqrt().item()

    print("RMS error between MLP and best-fit linear model =", error)
    return error
check_linear_behavior(model.backbone.cpu(), input_dim=512)

RMS error between MLP and best-fit linear model = 0.004029848147183657


0.004029848147183657